<a href="https://colab.research.google.com/github/jaragos0s/deep_learning/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

핸즈온 머신러닝 
#Chapter16. RNN과 어텐션을 사용한 자연어 처리

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Char-RNN을 사용해 셰익스피어 문체 생성하기

In [42]:
import sys
import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


In [43]:
'''작품 다운로드'''
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

In [44]:
print(shakespeare_text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [45]:
'''shakespeare.txt 파일에 어떤 문자가 있는지 sorting 해서 출력'''
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [46]:
''' 모든 글자를 정수로 인코딩 '''
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True) # char_level = True -> 단어 수준 인코딩(기본적으로 텍스트를 소문자로 바꿈)
tokenizer.fit_on_texts(shakespeare_text)

In [47]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [48]:
max_id = len(tokenizer.word_index) # 고유 글자 개수
dataset_size = tokenizer.document_count # 전체 글자 개수

In [49]:
''' 전체 텍스트를 인코딩해 각 글자를 ID로 나타냄 '''
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

### 순차 데이터셋 나누기

훈련 세트, 검증 세트, 테스트 세트가 중복되지 않도록 만들어야 한다. 

시계열 데이터는 보통 시간에 따라 나눈다. 


In [50]:
''' 텍스트의 처음 90%를 훈련 세트로 사용 '''
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### 순차 데이터를 윈도 여러 개로 자르기

데이터셋의 window() 메서드를 사용해 긴 시퀀스를 작은, 많은 텍스트 윈도(매우 짧은 부분 문자열)로 변환

RNN은 이 부분 문자열 길이만큼만 역전파를 위해 펼쳐진다. -> TBPTT

In [51]:
n_steps = 100
window_length = n_steps + 1 # target = 1글자 앞의 input

In [52]:
dataset = dataset.window(window_length, shift = 1, drop_remainder = True) 
#shift = 1 : 가장 큰 훈련세트를 만듦.
# drop_remainder = True : 모든 윈도가 동일하게 101개의 글자를 포함하도록 설정

- window() : 각각 하나의 데이터셋으로 표현되는 윈도를 포함하는 데이터셋 생성. -> nested dataset(데이터셋 메서드를 호출하여 각 윈도를 변환할 때 유용)

BUT 모델은 데이터셋이 아니라 "텐서"를 기대하기 때문에 훈련에 중첩 데이터셋을 바로 사용할 수 없음 -> flat dadtaset(데이터셋이 들어 있지 않는 데이터셋)으로 변환해야 함. 

- flat_map() : 중첩 데이터셋을 평평하게 만들기 전에 각 데이터셋에 적용할 변환 함수를 매개변수로 받을 수 있음. 

In [53]:
dataset = dataset.flat_map(lambda window:window.batch(window_length)) # window_length만큼 읽고 lambda 함수를 적용해 텐서 반환 

In [54]:
''' 결과값 유지 위해 시드 설정 '''
np.random.seed(42)
tf.random.set_seed(42)

In [55]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [56]:
''' 일반적으로 categorical 입력 특성은 one-hot vector 이나 embedding으로 인코딩 '''
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth = max_id), Y_batch))

In [57]:
# 데이터셋에 필요한 데이터를 미리 불러오기
''' CPU 연산속도 < 메모리 접근속도 가 되어 CPU가 놀게 되는 비효율적인 현상 방지하기 위해
    prefetch를 이용해 cache나 main memory에서 데이터를 찾음. '''
dataset = dataset.prefetch(1)

### Creating & Training the Char-RNN model

In [58]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# label이 정수형일 때 sparse_categorical_crossentropy 사용. 
# label이 one-hot vector일 때에는 categorical_crossentropy 사용
# 우리는 label을 one-hot-vector로 만든게 아니라서 sparse_categorical_crossentropy로 지정

'''아 짱 오래 걸림...... 오늘 안에 끝날까... 예측 하고 싶은데 ...'''

history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)

'아 짱 오래 걸림...... 오늘 안에 끝날까... 예측 하고 싶은데 ...'

### Char-RNN 모델 사용하기

In [61]:
'''전처리'''
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts)) - 1
  return tf.one_hot(X, max_id)

In [62]:
X_new = preprocess[("I love yo")]
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 첫번째 문장, 마지막 글자

TypeError: ignored

### 가짜 셰익스피어 텍스트 생성하기

In [63]:
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()
# we can select next character randomly based on the probablity that model predicts

array([[1, 1, 0, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 2, 0, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 1, 2, 0, 1, 0, 0, 1, 1, 2, 0, 0, 0, 2, 0, 0]])

In [64]:
def next_char(text, temperature = 1): # control the variety of generated texts
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numy())[0]

In [65]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text

### 상태가 있는 RNN

RNN이 한 훈련 배치를 처리한 후에 마지막 상태를 다음 훈련 배치의 초기 상태로 사용 

-> 역전파는 짧은 시퀀스에서 일어나지만 모델이 장기간 패턴을 학습할 수 있음

순차적이고 겹치지 않는 입력 시퀀스 생성(shift = n_steps) 
-> 하나의 윈도를 갖는 배치를 만들기


In [67]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift = n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth= max_id), Y_batch))
dataset = dataset.prefetch(1)

In [88]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

1. 각 순환층을 만들 때 stateful = True로 지정
2. 상태가 있는 RNN은 배치 크기를 알아야 함. -> 첫번째 층에 batch_input_shape 매개변수 지정

In [89]:
model = keras.models.Sequential([
                                 keras.layers.GRU(128, return_sequences=True, stateful = True, 
                                                  dropout = 0.2, recurrent_dropout = 0.2, 
                                                  batch_input_shape = [batch_size, None, max_id]),
                                 keras.layers.GRU(128, return_sequences=True, stateful = True, 
                                                  dropout = 0.2, recurrent_dropout = 0.2),
                                 keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [90]:
#epoch 끝마다 텍스트를 다시 시작하기 전에 상태 재설정
class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs = 50, callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 76s 236ms/step - loss: 2.8978
Epoch 2/50
313/313 [==============================] - 76s 242ms/step - loss: 2.2689
Epoch 3/50
313/313 [==============================] - 76s 243ms/step - loss: 2.5783
Epoch 4/50
313/313 [==============================] - 78s 250ms/step - loss: 2.6874
Epoch 5/50
313/313 [==============================] - 78s 249ms/step - loss: 2.4219
Epoch 6/50
313/313 [==============================] - 76s 242ms/step - loss: 2.0524
Epoch 7/50
313/313 [==============================] - 78s 249ms/step - loss: 2.0933
Epoch 8/50
313/313 [==============================] - 76s 243ms/step - loss: 2.1620
Epoch 9/50
313/313 [==============================] - 78s 248ms/step - loss: 2.3065
Epoch 10/50
313/313 [==============================] - 77s 247ms/step - loss: 2.2710
Epoch 11/50
313/313 [==============================] - 76s 244ms/step - loss: 2.2148
Epoch 12/50
313/313 [==============================] - 76s 244ms/step - lo